# Задачи коммивояжера методом ветвей и границ


In [1]:
"""
Для использования алгоритма с произволными значениями расстояний.
Требуется передать в класс tsp переменную matrix (матрицу расстояний)
По диагонали должны присутвствовать псевдобесконечное значение( Например: 999)
Пример:
    matrix = [[999,0,1],
              [0,999,1],
              [1,1,999]]

Для проверки алгоритма присутсвует тестируемая оболочка

За основу взят материал из источника: https://math.semestr.ru/kom/komm.php
Для проверки маршрута использовал онлайн калькулятор: https://math.semestr.ru/kom/index.php
"""
import copy

class tsp:
    def __init__(self):
        """
        Функция для инициализации переменных
        """
        self.price = 0
        
    def make_true(self, matrix):
        """
        Функция для перевода числовой матррицы в булевую


        Args:
            matrix - матрица
        Returns:
            matrix_bool - булевая матрица 
        """
        matrix_bool = copy.deepcopy(matrix)
        for i in range(len(matrix)):
            for j in range(len(matrix[0])):
                matrix_bool[i][j] = True
        return matrix_bool

    def min_row(self,matrix,matrix_bool):
        """
        Функция для поиска минимальных значений в строках

        Args:
            matrix - матрица 
            matrix_bool - матрица булевых значений 

        Returns:
            row - список минимальных значений в строке
        """
        row = []
        for i in range(len(matrix)):
            min_el = 9999
            for j in range(len(matrix[0])):
                if matrix_bool[i][j]:
                    if matrix[i][j] < min_el:
                        min_el = matrix[i][j]
            row.append(min_el)
        return row

    def min_row_without_way(self,matrix,matrix_bool):
        """
        Функция для поиска минимальных значений в строках без учета конкретного города.
        С проверкой на нулевое расстояние

        Args:
            matrix - матрица 
            matrix_bool - матрица булевых значений 

        Returns:
            row - список минимальных значений в строке
        """
        row = []
        for i in range(len(matrix)):
            min_el = 9999
            count_zero = 0
            for j in range(len(matrix[0])):
                if matrix_bool[i][j]:
                    if matrix[i][j] < min_el:
                        if matrix[i][j] == 0:
                            count_zero +=1
                        else:
                            min_el = matrix[i][j]
                        if count_zero >1:
                            min_el = 0 
            row.append(min_el)
        return row

    def min_column(self,matrix,matrix_bool):
        """
        Функция для поиска минимальных значений в колонках

        Args:
            matrix - матрица 
            matrix_bool - матрица булевых значений 

        Returns:
            column - список минимальных значений в колонке 
        """
        column = []
        for i in range(len(matrix)):
            min_el = 9999
            for j in range(len(matrix[0])):
                if matrix_bool[j][i]:
                    if matrix[j][i] < min_el:
                        min_el = matrix[j][i]
            column.append(min_el)
        return column

    def min_column_without_way(self,matrix,matrix_bool):
        """
        Функция для поиска минимальных значений в колонках без учета конкретного города.
        С проверкой на нулевое расстояние

        Args:
            matrix - матрица 
            matrix_bool - матрица булевых значений 

        Returns:
            column - список минимальных значений в колонке 
        """
        column = []
        for i in range(len(matrix)):
            min_el = 9999
            count_zero = 0
            for j in range(len(matrix[0])):
                if matrix_bool[j][i]:
                    if matrix[j][i] < min_el:
                        if matrix[j][i] == 0:
                            count_zero +=1
                        else:
                            min_el = matrix[j][i]
                        if count_zero > 1:
                            min_el = 0
            column.append(min_el)
        return column 

    def reduction_matrix_row(self,matrix,row,matrix_bool):
        """
        Функция для редукции матрицы по строкам путём вычитания минимального значения

        Args:
            matrix - матрица 
            row - список минимальных значений в строках
            matrix_bool - матрица булевых значений 

        Returns:
            matrix - сокращенная матрица по строкам
        """
        for i in range(len(matrix)):
            for j in range(len(matrix[0])):
                if matrix_bool[i][j]:
                    matrix[i][j] -= row[i]
        return matrix

    def reduction_matrix_column(self,matrix,column,matrix_bool):
        """
        Функция для редукции матрицы по столбцам путём вычитания минимального значения

        Args:
            matrix - матрица 
            column - список минимальных значений в столбцах
            matrix_bool - матрица булевых значений 

        Returns:
            matrix - сокращенная матрица по столбцам 
        """
        for i in range(len(matrix)):
            for j in range(len(matrix[0])):
                if matrix_bool[j][i]:
                    matrix[j][i] -= column[i]
        return matrix

    def find_zero(self,matrix,matrix_bool):
        """
        Функция для поиска всех клеток матрицы с нулевыми элементами

        Args:
            matrix - матрица
            matrix_bool - матрица булевых значений  
        Returns:
            zero_list - список координат клеток с нулевыми элементами
        """
        zero_list = []
        for i in range(len(matrix)):
            for j in range(len(matrix[0])):
                if matrix_bool[i][j]:
                    if matrix[i][j] == 0:
                        zero_list.append((i,j))
        return zero_list

    def find_max_tree(self,matrix,row,column,zero_list):
        """
        Функция для поиска самой большой цены маршрута

        Args:
            matrix - матрица
            matrix_bool - матрица булевых значений 
            zero_list - список координат клеток с нулевыми элементами
            row - список минимальных значений в строках
            column - список минимальных значений в столбцах
        Returns:
            max_way - координата клетки, где максимальная цена 
        """
        count_const = {}
        for i in zero_list:
            count_const[i] = row[i[0]] + column[i[1]]
        max_way = max(count_const, key=count_const.get)
        return max_way

    def delete_tree(self,matrix_bool,max_way):
        """
        Функция для замены на значение False в булевой матрице удаленные(использованные) строки и столбцы  

        Args:
            matrix_bool - булевая матрица
            max_way - координата клетки, у которой нужно удалить столбец и строку
        Returns:
             matrix_bool - булевая матрица
        """
        for j in range(len(self.matrix[max_way[0]])):
            matrix_bool[max_way[0]][j] = False
        for j in range(len(self.matrix[max_way[1]])):
            matrix_bool[j][max_way[1]] = False

        return matrix_bool

    def change_zero(self,matrix,way):
        """
        Функция для замены использованного пути на псевдобесконечное число 

        Args:
            matrix - матрица
            way - координата клетки
        Returns:
             matrix - матрица с измененным значеним 
        """
        matrix[way[0]][way[1]] = 999
        return matrix

    def Reverse(self,tuples): 
        """
        Функция для разворота кортежи

        Args:
            tuples - кортеж
        Returns:
             new_tup - новый кортеж
        """
        new_tup = tuples[::-1] 
        return new_tup 

    def sort_way(self,way):
        """
        Функция для сортировки пути

        Args:
            way - неотсортированный путь 
        Returns:
             sorted_way - отсортированный путь 
        """
        sorted_way = []
        second_element = way[0][1]
        for i in range(len(way)):
            for j in way:
                if second_element == j[0]:
                    sorted_way.append(j)
                    second_element = j[1]
                    break
        return sorted_way

    def find_price_ways(self,matrix,way):
        """
        Функция для нахождения суммы пути
        Args:
            matrix - матрица
            way - путь 
        Returns:
            price - сумма пути 
        """
        price = 0
        for i in way:
            price += matrix[i[0]][i[1]]
        return price
    
    
    def start(self,matrix):
        """
        Функция для старта алгоритма, печатает путь и длину маршрута
        Args:
            matrix - матрица 
        """
        matrix = matrix
        matrix_clear = copy.deepcopy(matrix)
        # создается булевая матрица, для запрета на использования удаленныйх элементов 
        matrix_bool = self.make_true(matrix)
        way_full = []
        for i in range(len(matrix_clear)):
            # Найдём минимальные значения по строкам
            row = self.min_row(matrix,matrix_bool)
            # Производим редукцию по строкам путём вычитания минимального значения
            self.matrix = self.reduction_matrix_row(matrix,row,matrix_bool)
            #Найдём минимальные значения по столбцам 
            column = self.min_column(matrix,matrix_bool)
            # Аналогично производим редукцию по столбцам.
            self.matrix = self.reduction_matrix_column(matrix,column,matrix_bool)
            # В получившейся матрице таблице находим нулевые значения
            zero_list = self.find_zero(matrix,matrix_bool)
            # находим минимальные элементы в строках и столбац без учета использованных маршрутов
            row = self.min_row_without_way(matrix,matrix_bool)
            column = self.min_column_without_way(matrix,matrix_bool)
            # Вычисляем оценку для нулевых значений
            max_way = self.find_max_tree(matrix,row,column,zero_list)

            #Сократим матрицу потерь за счёт исключения строки и столбца с наибольшей оценкой
            matrix_bool = self.delete_tree(matrix_bool,max_way)
            # необходимо вместо нуля (минимального значения) поставить псевдобесконечное число 
            self.matrix = self.change_zero(matrix,self.Reverse(max_way))
            # Добавляем все пути
            way_full.append(max_way)
        # Находим отсортированный путь
        way = self.sort_way(way_full)   
        # Находим сумму процденного расстояния 
        self.price = self.find_price_ways(matrix_clear,way)
        print('Путь {} Пройденное расстояние = {}'.format(way,self.price))



In [2]:
class test:
    def first_test():
        way = 172
        matrix = [[999,41,40,48,40,42],
                  [48,999,41,49,42,46],
                  [22,22,999,23,24,19],
                  [15,17,11,999,10,14],
                  [47,43,18,42,999,52],
                  [34,39,30,39,32,999]]
        print('\nДлина пути должна быть равна ', way)
        class_tsp = tsp()
        class_tsp.start(matrix)
        if class_tsp.price == way:
            print('Первый тест пройден')
        else:
            print('Первый тест завален')
    def second_test():
        way = 180
        matrix = [[999,90,80,40,100],
          [60,999,40,50,70],
          [50,30,999,60,20],
          [10,70,20,999,50],
          [20,40,50,20,999]]
        print('\nДлина пути должна быть равна ', way)
        class_tsp = tsp()
        class_tsp.start(matrix)
        if class_tsp.price == way:
            print('Второй тест пройден')
        else:
            print('Второй тест завален')
    def three_test():
        way = 85
        matrix = [[999,10,15,11,2,55],
                  [17,999,16,18,21,13],
                  [10,50,999,39,22,3],
                  [28,29,24,999,28,25],
                  [27,9,32,9,999,2],
                  [43,48,40,43,21,999]]
        print('\nДлина пути должна быть равна ', way)
        class_tsp = tsp()
        class_tsp.start(matrix)
        if class_tsp.price == way:
            print('Третий тест пройден')
        else:
            print('Третий тест завален')
    def four_test():
        way = 77
        matrix = [[999,0,15,11,2,55],
                  [17,999,16,18,21,13],
                  [10,50,999,39,22,3],
                  [28,29,24,999,28,25],
                  [27,9,32,9,999,2],
                  [43,48,40,43,21,999]]
        print('\nДлина пути должна быть равна ', way)
        class_tsp = tsp()
        class_tsp.start(matrix)
        if class_tsp.price == way:
            print('Четвертый тест пройден')
        else:
            print('Четвертый тест завален')
    def five_test():
        way = 2
        matrix = [[999,0,1],
                  [0,999,1],
                  [1,1,999]]
        print('\nДлина пути должна быть равна ', way)
        class_tsp = tsp()
        class_tsp.start(matrix)
        if class_tsp.price == way:
            print('Пятый тест пройден')
        else:
            print('Пятый тест завален')
    def six_test():
        way = 30
        matrix = [[999,5,11,9],
                  [20,999,8,7],
                  [7,14,999,8],
                 [12,6,15,999]]
        print('\nДлина пути должна быть равна ', way)
        class_tsp = tsp()
        class_tsp.start(matrix)
        if class_tsp.price == way:
            print('Шестой тест пройден')
        else:
            print('Шестой тест завален')
    def seven_test():
        way = 1
        matrix = [[999,0,0],
                  [1,999,1],
                  [1,0,999]]
        print('\nДлина пути должна быть равна ', way)
        class_tsp = tsp()
        class_tsp.start(matrix)
        if class_tsp.price == way:
            print('Седьмой тест пройден')
        else:
            print('Седьмой тест завален \n')
    def eight_test():
        way = 0
        matrix = [[999,0,0],
                  [0,999,0],
                  [0,0,999]]
        print('\nДлина пути должна быть равна ', way)
        class_tsp = tsp()
        class_tsp.start(matrix)
        if class_tsp.price == way:
            print('Восьмой тест пройден')
        else:
            print('Восьмой тест завален \n')
    
    
    if  __name__ == '__main__':
        first_test()
        second_test()
        three_test()
        four_test()
        five_test()
        six_test()
        seven_test()
        eight_test()
test()


Длина пути должна быть равна  172
Путь [(2, 3), (3, 5), (5, 0), (0, 1), (1, 4), (4, 2)] Пройденное расстояние = 172
Первый тест пройден

Длина пути должна быть равна  180
Путь [(3, 2), (2, 4), (4, 1), (1, 0), (0, 3)] Пройденное расстояние = 180
Второй тест пройден

Длина пути должна быть равна  85
Путь [(4, 1), (1, 0), (0, 3), (3, 2), (2, 5), (5, 4)] Пройденное расстояние = 85
Третий тест пройден

Длина пути должна быть равна  77
Путь [(4, 3), (3, 2), (2, 0), (0, 1), (1, 5), (5, 4)] Пройденное расстояние = 77
Четвертый тест пройден

Длина пути должна быть равна  2
Путь [(1, 2), (2, 0), (0, 1)] Пройденное расстояние = 2
Пятый тест пройден

Длина пути должна быть равна  30
Путь [(0, 3), (3, 1), (1, 2), (2, 0)] Пройденное расстояние = 30
Шестой тест пройден

Длина пути должна быть равна  1
Путь [(0, 2), (2, 1), (1, 0)] Пройденное расстояние = 1
Седьмой тест пройден

Длина пути должна быть равна  0
Путь [(1, 2), (2, 0), (0, 1)] Пройденное расстояние = 0
Восьмой тест пройден
